In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

# Mount Google Drive to access data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define data paths
macro_folder = '/content/drive/MyDrive/processed_data'
financial_folder = '/content/drive/MyDrive/financial_info'


Mounted at /content/drive


In [2]:
def load_and_preprocess_data(country, include_macro=True):
    """
    Load and preprocess data for a specific country

    Parameters:
    -----------
    country : str
        Country name (brazil, india, south_africa)
    include_macro : bool
        Whether to include macroeconomic factors

    Returns:
    --------
    processed_df : DataFrame
        Processed data with market regimes
    """
    print(f"Processing {country.upper()} data...")

    # Load data
    stock_file = f'{financial_folder}/{country}_stock.csv'
    macro_file = f'{macro_folder}/{country}_macro.csv'

    # Load stock data - skip ticker symbol row
    stock_df = pd.read_csv(stock_file, parse_dates=['Date'], skiprows=[1])

    # Standardize column names to lowercase
    stock_df.columns = [col.lower() for col in stock_df.columns]
    if 'date' not in stock_df.columns and 'Date' in stock_df.columns:
        stock_df = stock_df.rename(columns={'Date': 'date'})

    # Convert close price to numeric
    stock_df['close'] = pd.to_numeric(stock_df['close'], errors='coerce')

    # Calculate returns at different frequencies
    stock_df['return_1m'] = stock_df['close'].pct_change(1)
    stock_df['return_3m'] = stock_df['close'].pct_change(3)
    stock_df['volatility'] = stock_df['return_1m'].rolling(20).std()

    # Calculate quarterly metrics
    stock_df['return_1q'] = stock_df['close'].pct_change(63)  # ~63 trading days in a quarter
    stock_df['return_2q'] = stock_df['close'].pct_change(126)  # ~126 trading days in 2 quarters
    stock_df['volatility_2q'] = stock_df['return_1q'].rolling(126).std()

    processed_df = stock_df.copy()

    if include_macro:
        # Load and process macro data
        macro_df = pd.read_csv(macro_file, parse_dates=['date'])
        macro_df.columns = [col.lower() for col in macro_df.columns]

        # Remove duplicate columns (keep only columns without suffix)
        macro_cols_to_keep = [col for col in macro_df.columns
                             if not col.endswith('_x') and not col.endswith('_y') and col != 'date']

        # Keep only unique columns
        macro_df = macro_df[['date'] + macro_cols_to_keep]

        # Convert all macro columns to numeric
        for col in macro_cols_to_keep:
            macro_df[col] = pd.to_numeric(macro_df[col], errors='coerce')

        # Forward fill missing values in macro data
        macro_df = macro_df.sort_values('date').set_index('date')
        macro_df = macro_df.ffill()
        macro_df = macro_df.reset_index()

        # Merge stock and macro data
        processed_df = pd.merge_asof(stock_df.sort_values('date'),
                                    macro_df.sort_values('date'), on='date')

        # Calculate percentage changes for macro columns
        for col in macro_cols_to_keep:
            if col in processed_df.columns:
                processed_df[f'{col}_change'] = processed_df[col].pct_change(1).fillna(0)

    # Define market regimes
    processed_df = define_market_regimes(processed_df)

    # Fill missing values
    for col in processed_df.columns:
        if col != 'date' and col != 'country' and col != 'regime':
            if col.endswith('_change'):
                processed_df[col] = processed_df[col].fillna(0)
            else:
                processed_df[col] = processed_df[col].ffill().bfill()

    processed_df['country'] = country

    print(f"Successfully processed {country} data: {processed_df.shape[0]} rows, {processed_df.shape[1]} columns")
    print(f"Date range: {processed_df['date'].min()} to {processed_df['date'].max()}")

    return processed_df


In [3]:
def define_market_regimes(df):
    """
    Define market regimes based on returns and volatility

    Parameters:
    -----------
    df : DataFrame
        DataFrame with return and volatility data

    Returns:
    --------
    df : DataFrame
        DataFrame with market regimes
    """
    df_copy = df.copy()

    # Use quarterly metrics for regime definition
    mask = df_copy['return_1q'].notna() & df_copy['volatility_2q'].notna()
    valid_data = df_copy[mask]

    if len(valid_data) == 0:
        # Fall back to monthly metrics if quarterly not available
        mask = df_copy['return_1m'].notna() & df_copy['volatility'].notna()
        valid_data = df_copy[mask]

    # Calculate quantiles only on valid data
    return_high = valid_data['return_1q'].quantile(0.7) if 'return_1q' in valid_data.columns else valid_data['return_1m'].quantile(0.7)
    return_low = valid_data['return_1q'].quantile(0.3) if 'return_1q' in valid_data.columns else valid_data['return_1m'].quantile(0.3)
    vol_high = valid_data['volatility_2q'].quantile(0.7) if 'volatility_2q' in valid_data.columns else valid_data['volatility'].quantile(0.7)

    # Define conditions for regimes (0: Bear, 1: Sideways, 2: Bull)
    if 'return_1q' in df_copy.columns and 'volatility_2q' in df_copy.columns:
        conditions = [
            (df_copy['return_1q'] < return_low) & (df_copy['volatility_2q'] > vol_high),
            (df_copy['return_1q'] >= return_low) & (df_copy['return_1q'] <= return_high),
            (df_copy['return_1q'] > return_high)
        ]
    else:
        conditions = [
            (df_copy['return_1m'] < return_low) & (df_copy['volatility'] > vol_high),
            (df_copy['return_1m'] >= return_low) & (df_copy['return_1m'] <= return_high),
            (df_copy['return_1m'] > return_high)
        ]

    choices = [0, 1, 2]  # 0: Bear, 1: Sideways, 2: Bull

    # Apply conditions to create regime column
    df_copy['regime'] = np.select(conditions, choices, default=1)

    return df_copy


In [4]:
def prepare_features(df, include_macro=True):
    """
    Prepare features for model training

    Parameters:
    -----------
    df : DataFrame
        Processed data with market regimes
    include_macro : bool
        Whether to include macroeconomic factors

    Returns:
    --------
    X : DataFrame
        Feature matrix
    y : Series
        Target variable (market regimes)
    feature_names : list
        List of feature names
    """
    # Base features (always included)
    base_features = ['return_1q', 'return_2q', 'volatility_2q']

    # Macro features (included if requested)
    macro_features = []
    if include_macro:
        possible_macro = ['gdp', 'cpi', 'exchange_rate', 'imports', 'exports',
                         'gdp_change', 'cpi_change', 'imports_change', 'exports_change']
        macro_features = [f for f in possible_macro if f in df.columns]

    # Combine features
    feature_names = base_features + macro_features

    # Select only available features
    available_features = [f for f in feature_names if f in df.columns]

    # Prepare feature matrix and target variable
    X = df[available_features].copy()
    y = df['regime'].copy()

    return X, y, available_features


In [5]:
def build_regime_classification_model(X_train, y_train):
    """
    Build and train a Support Vector Classifier for market regime classification

    Parameters:
    -----------
    X_train : DataFrame
        Training features
    y_train : Series
        Target variable (market regimes)

    Returns:
    --------
    model : Pipeline
        Trained classification model with preprocessing steps
    """

    # Check for NaN values in training data
    if X_train.isna().any().any():
        print(f"Warning: Training data contains {X_train.isna().sum().sum()} NaN values")

    # Create pipeline with preprocessing and model
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('scaler', StandardScaler()),                 # Scale features
        ('classifier', SVC(C=1.0, gamma='scale', probability=True))  # SVM classifier
    ])

    # Train the model
    pipeline.fit(X_train, y_train)

    return pipeline


In [6]:
def train_traditional_models(X_train, X_test, y_train, y_test, feature_names):
    """
    Train and evaluate traditional ML models (Random Forest, XGBoost, SVC)

    Parameters:
    -----------
    X_train, X_test : DataFrame
        Training and test feature matrices
    y_train, y_test : Series
        Training and test target variables
    feature_names : list
        List of feature names

    Returns:
    --------
    results : dict
        Dictionary of model results
    """
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize models
    models = {
        'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42),
        'XGBoost': XGBClassifier(n_estimators=200, learning_rate=0.1, random_state=42),
        'SVC': SVC(probability=True, random_state=42)
    }

    results = {}

    for name, model in models.items():
        print(f"Training {name}...")

        # Train model
        model.fit(X_train_scaled, y_train)

        # Make predictions
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)

        # Calculate metrics
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)

        # Store results
        results[name] = {
            'model': model,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': f1_score(y_train, y_train_pred, average='weighted'),
            'test_f1': f1_score(y_test, y_test_pred, average='weighted'),
            'confusion_matrix': confusion_matrix(y_test, y_test_pred),
            'y_pred': y_test_pred,
            'y_true': y_test
        }

        # Get feature importance for tree-based models
        if name in ['RandomForest', 'XGBoost']:
            results[name]['feature_importance'] = dict(zip(feature_names, model.feature_importances_))

        print(f"{name} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return results

def create_sequences(data, target, seq_length=10):
    """Create sequences for RNN models"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(target[i+seq_length])
    return np.array(X), np.array(y)

def train_deep_learning_models(X_train, X_test, y_train, y_test, feature_names):
    """
    Train and evaluate deep learning models (RNN, LSTM, GRU)

    Parameters:
    -----------
    X_train, X_test : DataFrame
        Training and test feature matrices
    y_train, y_test : Series
        Training and test target variables
    feature_names : list
        List of feature names

    Returns:
    --------
    results : dict
        Dictionary of model results
    """
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create sequences for RNN models
    seq_length = 10
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train.values, seq_length)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test.values, seq_length)

    # One-hot encode target
    y_train_onehot = tf.keras.utils.to_categorical(y_train_seq)
    y_test_onehot = tf.keras.utils.to_categorical(y_test_seq)

    # Define model architectures
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    output_shape = y_train_onehot.shape[1]

    models = {
        'SimpleRNN': Sequential([
            SimpleRNN(64, input_shape=input_shape, return_sequences=True),
            Dropout(0.2),
            SimpleRNN(32),
            Dropout(0.2),
            Dense(output_shape, activation='softmax')
        ]),
        'LSTM': Sequential([
            LSTM(64, input_shape=input_shape, return_sequences=True),
            Dropout(0.2),
            LSTM(32),
            Dropout(0.2),
            Dense(output_shape, activation='softmax')
        ]),
        'GRU': Sequential([
            GRU(64, input_shape=input_shape, return_sequences=True),
            Dropout(0.2),
            GRU(32),
            Dropout(0.2),
            Dense(output_shape, activation='softmax')
        ])
    }

    results = {}

    for name, model in models.items():
        print(f"Training {name}...")

        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        # Train model
        history = model.fit(
            X_train_seq, y_train_onehot,
            epochs=50,
            batch_size=32,
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=0
        )

        # Make predictions
        y_train_pred_prob = model.predict(X_train_seq)
        y_test_pred_prob = model.predict(X_test_seq)

        y_train_pred = np.argmax(y_train_pred_prob, axis=1)
        y_test_pred = np.argmax(y_test_pred_prob, axis=1)

        y_train_true = np.argmax(y_train_onehot, axis=1)
        y_test_true = np.argmax(y_test_onehot, axis=1)

        # Calculate metrics
        train_accuracy = accuracy_score(y_train_true, y_train_pred)
        test_accuracy = accuracy_score(y_test_true, y_test_pred)

        # Store results
        results[name] = {
            'model': model,
            'train_accuracy': train_accuracy,
            'test_accuracy': test_accuracy,
            'train_f1': f1_score(y_train_true, y_train_pred, average='weighted'),
            'test_f1': f1_score(y_test_true, y_test_pred, average='weighted'),
            'confusion_matrix': confusion_matrix(y_test_true, y_test_pred),
            'history': history.history,
            'y_pred': y_test_pred,
            'y_true': y_test_true
        }

        print(f"{name} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return results



In [7]:
def simulate_portfolio(df, predictions, start_date, end_date):
    """
    Simulate portfolio performance based on model predictions

    Parameters:
    -----------
    df : DataFrame
        Original data with market prices
    predictions : dict
        Dictionary of model predictions
    start_date : str
        Start date for simulation
    end_date : str
        End date for simulation

    Returns:
    --------
    portfolio_returns : dict
        Dictionary of portfolio returns
    """
    # Filter data for simulation period
    sim_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()
    sim_df = sim_df.sort_values('date')

    # Calculate daily returns
    sim_df['daily_return'] = sim_df['close'].pct_change()

    # Initialize portfolio returns
    portfolio_returns = {
        'Buy and Hold': [],
        **{model_name: [] for model_name in predictions.keys()}
    }

    # Calculate cumulative returns
    cum_return_bh = 1.0
    cum_returns_models = {model_name: 1.0 for model_name in predictions.keys()}

    # Simulate portfolio performance
    for i in range(1, len(sim_df)):
        date = sim_df.iloc[i]['date']
        daily_return = sim_df.iloc[i]['daily_return']

        # Buy and Hold strategy
        if not np.isnan(daily_return):
            cum_return_bh *= (1 + daily_return)
            portfolio_returns['Buy and Hold'].append({
                'date': date,
                'cumulative_return': cum_return_bh
            })

        # Model-based strategies
        for model_name, model_preds in predictions.items():
            # Get the predicted regime for the current date
            if i-1 < len(model_preds):
                pred_regime = model_preds[i-1]

                # Investment strategy based on regime
                # 0: Bear - Short the market (-1x exposure)
                # 1: Sideways - Cash (0x exposure)
                # 2: Bull - Long the market (1x exposure)
                exposure = -1.0 if pred_regime == 0 else (0.0 if pred_regime == 1 else 1.0)

                if not np.isnan(daily_return):
                    model_return = daily_return * exposure
                    cum_returns_models[model_name] *= (1 + model_return)
                    portfolio_returns[model_name].append({
                        'date': date,
                        'cumulative_return': cum_returns_models[model_name]
                    })

    # Convert to DataFrames
    for strategy, returns in portfolio_returns.items():
        portfolio_returns[strategy] = pd.DataFrame(returns)

    return portfolio_returns


In [8]:
def evaluate_portfolio_performance(portfolio_returns):
    """
    Calculate performance metrics for portfolios

    Parameters:
    -----------
    portfolio_returns : dict
        Dictionary of portfolio returns

    Returns:
    --------
    performance_metrics : DataFrame
        DataFrame of performance metrics
    """
    performance = {}

    for strategy, returns_df in portfolio_returns.items():
        # Calculate daily returns
        returns_df['daily_return'] = returns_df['cumulative_return'].pct_change()

        # Calculate performance metrics
        total_return = returns_df['cumulative_return'].iloc[-1] / returns_df['cumulative_return'].iloc[0] - 1
        annual_return = (1 + total_return) ** (252 / len(returns_df)) - 1
        volatility = returns_df['daily_return'].std() * np.sqrt(252)
        sharpe_ratio = annual_return / volatility if volatility > 0 else 0

        # Calculate maximum drawdown
        cumulative = returns_df['cumulative_return']
        running_max = cumulative.cummax()
        drawdown = (cumulative / running_max - 1)
        max_drawdown = drawdown.min()

        performance[strategy] = {
            'Total Return': total_return,
            'Annual Return': annual_return,
            'Volatility': volatility,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown': max_drawdown
        }

    return pd.DataFrame(performance).T


In [9]:
def run_market_regime_simulation(countries=['brazil', 'india', 'south_africa']):
    """
    Run market regime simulation for multiple countries

    Parameters:
    -----------
    countries : list
        List of countries to simulate

    Returns:
    --------
    results : dict
        Dictionary of simulation results
    """
    simulation_results = {}

    for country in countries:
        print(f"\n{'='*50}")
        print(f"SIMULATING {country.upper()} MARKET REGIMES")
        print(f"{'='*50}")

        # Load data with and without macro factors
        data_with_macro = load_and_preprocess_data(country, include_macro=True)
        data_without_macro = load_and_preprocess_data(country, include_macro=False)

        # Split data into training and testing periods
        # Use first 70% for training, last 30% for testing
        train_cutoff = int(len(data_with_macro) * 0.7)

        train_data_with_macro = data_with_macro.iloc[:train_cutoff]
        test_data_with_macro = data_with_macro.iloc[train_cutoff:]

        train_data_without_macro = data_without_macro.iloc[:train_cutoff]
        test_data_without_macro = data_without_macro.iloc[train_cutoff:]

        # Prepare features
        X_train_with_macro, y_train_with_macro, features_with_macro = prepare_features(
            train_data_with_macro, include_macro=True)
        X_test_with_macro, y_test_with_macro, _ = prepare_features(
            test_data_with_macro, include_macro=True)

        X_train_without_macro, y_train_without_macro, features_without_macro = prepare_features(
            train_data_without_macro, include_macro=False)
        X_test_without_macro, y_test_without_macro, _ = prepare_features(
            test_data_without_macro, include_macro=False)

        # Train models with macro factors
        print("\nTraining models WITH macroeconomic factors:")
        traditional_results_with_macro = train_traditional_models(
            X_train_with_macro, X_test_with_macro,
            y_train_with_macro, y_test_with_macro,
            features_with_macro)

        dl_results_with_macro = train_deep_learning_models(
            X_train_with_macro, X_test_with_macro,
            y_train_with_macro, y_test_with_macro,
            features_with_macro)

        # Train models without macro factors
        print("\nTraining models WITHOUT macroeconomic factors:")
        traditional_results_without_macro = train_traditional_models(
            X_train_without_macro, X_test_without_macro,
            y_train_without_macro, y_test_without_macro,
            features_without_macro)

        dl_results_without_macro = train_deep_learning_models(
            X_train_without_macro, X_test_without_macro,
            y_train_without_macro, y_test_without_macro,
            features_without_macro)

        # Combine results
        all_results_with_macro = {**traditional_results_with_macro, **dl_results_with_macro}
        all_results_without_macro = {**traditional_results_without_macro, **dl_results_without_macro}

        # Extract predictions for portfolio simulation
        predictions_with_macro = {
            f"{model_name}_with_macro": results['y_pred']
            for model_name, results in all_results_with_macro.items()
        }

        predictions_without_macro = {
            f"{model_name}_without_macro": results['y_pred']
            for model_name, results in all_results_without_macro.items()
        }

        # Combine predictions
        all_predictions = {**predictions_with_macro, **predictions_without_macro}

        # Run portfolio simulation
        start_date = test_data_with_macro['date'].min()
        end_date = test_data_with_macro['date'].max()

        portfolio_returns = simulate_portfolio(
            data_with_macro, all_predictions, start_date, end_date)

        # Evaluate portfolio performance
        performance_metrics = evaluate_portfolio_performance(portfolio_returns)

        # Store results
        simulation_results[country] = {
            'data_with_macro': data_with_macro,
            'data_without_macro': data_without_macro,
            'model_results_with_macro': all_results_with_macro,
            'model_results_without_macro': all_results_without_macro,
            'portfolio_returns': portfolio_returns,
            'performance_metrics': performance_metrics
        }

        # Print performance metrics
        print("\nPortfolio Performance Metrics:")
        print(performance_metrics)

        # Visualize portfolio performance
        plt.figure(figsize=(12, 6))
        for strategy, returns_df in portfolio_returns.items():
            plt.plot(returns_df['date'], returns_df['cumulative_return'], label=strategy)

        plt.title(f'{country.capitalize()} - Portfolio Performance')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Visualize feature importance for traditional models
        for model_name, results in traditional_results_with_macro.items():
            if 'feature_importance' in results:
                plt.figure(figsize=(10, 6))
                importance_df = pd.DataFrame({
                    'Feature': list(results['feature_importance'].keys()),
                    'Importance': list(results['feature_importance'].values())
                }).sort_values('Importance', ascending=False)

                sns.barplot(x='Importance', y='Feature', data=importance_df)
                plt.title(f'{country.capitalize()} - {model_name} Feature Importance (with Macro)')
                plt.tight_layout()
                plt.show()

    return simulation_results


In [10]:
def analyze_macro_value_added(simulation_results):
    """
    Analyze the value added by macroeconomic factors

    Parameters:
    -----------
    simulation_results : dict
        Dictionary of simulation results

    Returns:
    --------
    macro_value_analysis : DataFrame
        DataFrame of macro value analysis
    """
    macro_value_analysis = {}

    for country, results in simulation_results.items():
        # Extract performance metrics
        performance = results['performance_metrics']

        # Compare models with and without macro
        model_pairs = []
        for col in performance.index:
            if '_with_macro' in col:
                base_name = col.replace('_with_macro', '')
                without_macro = f"{base_name}_without_macro"
                if without_macro in performance.index:
                    model_pairs.append((col, without_macro))

        # Calculate value added
        for with_macro, without_macro in model_pairs:
            model_name = with_macro.replace('_with_macro', '')

            # Calculate differences
            sharpe_diff = performance.loc[with_macro, 'Sharpe Ratio'] - performance.loc[without_macro, 'Sharpe Ratio']
            return_diff = performance.loc[with_macro, 'Annual Return'] - performance.loc[without_macro, 'Annual Return']
            drawdown_diff = performance.loc[with_macro, 'Max Drawdown'] - performance.loc[without_macro, 'Max Drawdown']

            macro_value_analysis[f"{country}_{model_name}"] = {
                'Country': country,
                'Model': model_name,
                'Sharpe Ratio Improvement': sharpe_diff,
                'Return Improvement': return_diff,
                'Drawdown Improvement': drawdown_diff,
                'Value Added': 'Positive' if sharpe_diff > 0 else 'Negative'
            }

    return pd.DataFrame(macro_value_analysis).T


In [11]:
def analyze_regime_transitions(simulation_results):
    """
    Analyze regime transitions and their impact on performance

    Parameters:
    -----------
    simulation_results : dict
        Dictionary of simulation results

    Returns:
    --------
    transition_analysis : dict
        Dictionary of transition analysis
    """
    transition_analysis = {}

    for country, results in simulation_results.items():
        data = results['data_with_macro']

        # Identify regime transitions
        data['regime_shift'] = data['regime'].diff().abs() > 0

        # Calculate returns around transitions
        pre_transition_returns = []
        post_transition_returns = []

        transition_indices = data[data['regime_shift']].index

        for idx in transition_indices:
            if idx > 5 and idx < len(data) - 5:
                pre_return = data.iloc[idx-5:idx]['return_1m'].mean()
                post_return = data.iloc[idx:idx+5]['return_1m'].mean()

                pre_transition_returns.append(pre_return)
                post_transition_returns.append(post_return)

        # Calculate statistics
        transition_analysis[country] = {
            'Number of Transitions': len(transition_indices),
            'Avg Pre-Transition Return': np.mean(pre_transition_returns),
            'Avg Post-Transition Return': np.mean(post_transition_returns),
            'Return Differential': np.mean(post_transition_returns) - np.mean(pre_transition_returns)
        }

    return transition_analysis


In [12]:
# Run the simulation
simulation_results = run_market_regime_simulation()

# Analyze the value added by macroeconomic factors
macro_value_analysis = analyze_macro_value_added(simulation_results)
print("\nValue Added by Macroeconomic Factors:")
print(macro_value_analysis)

# Analyze regime transitions
transition_analysis = analyze_regime_transitions(simulation_results)
print("\nRegime Transition Analysis:")
for country, analysis in transition_analysis.items():
    print(f"\n{country.upper()}:")
    for metric, value in analysis.items():
        print(f"  {metric}: {value:.4f}" if isinstance(value, float) else f"  {metric}: {value}")



SIMULATING BRAZIL MARKET REGIMES
Processing BRAZIL data...
Successfully processed brazil data: 148 rows, 26 columns
Date range: 2013-01-01 00:00:00 to 2025-04-01 00:00:00
Processing BRAZIL data...
Successfully processed brazil data: 148 rows, 14 columns
Date range: 2013-01-01 00:00:00 to 2025-04-01 00:00:00

Training models WITH macroeconomic factors:
Training RandomForest...
RandomForest - Train Accuracy: 1.0000, Test Accuracy: 1.0000
Training XGBoost...


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [1 2]

In [ ]:
def generate_simulation_report(simulation_results, macro_value_analysis, transition_analysis):
    """
    Generate a comprehensive report of simulation results
    """
    print("\n" + "="*80)
    print(" "*30 + "MARKET REGIME SIMULATION REPORT")
    print("="*80 + "\n")

    print("RESEARCH QUESTION:")
    print("To what extent can the identification and classification of market regimes in Brazil,")
    print("India, and South Africa contribute to improved portfolio performance and risk-adjusted")
    print("returns for hedge fund managers?\n")

    # Overall findings
    print("1. OVERALL FINDINGS")
    print("-"*80)

    # Calculate average improvement across all models and countries
    avg_sharpe_improvement = macro_value_analysis['Sharpe Ratio Improvement'].mean()
    avg_return_improvement = macro_value_analysis['Return Improvement'].mean()

    print(f"Average Sharpe Ratio Improvement with Macro Factors: {avg_sharpe_improvement:.4f}")
    print(f"Average Return Improvement with Macro Factors: {avg_return_improvement:.4f}")

    positive_value_pct = (macro_value_analysis['Value Added'] == 'Positive').mean() * 100
    print(f"Percentage of Models Showing Positive Value from Macro Factors: {positive_value_pct:.1f}%\n")

    # Best performing models
    best_model_idx = macro_value_analysis['Sharpe Ratio Improvement'].idxmax()
    best_model = macro_value_analysis.loc[best_model_idx]

    print(f"Best Performing Model: {best_model['Model']} in {best_model['Country'].capitalize()}")
    print(f"  Sharpe Ratio Improvement: {best_model['Sharpe Ratio Improvement']:.4f}")
    print(f"  Return Improvement: {best_model['Return Improvement']:.4f}\n")

    # Country-specific findings
    print("2. COUNTRY-SPECIFIC FINDINGS")
    print("-"*80)

    for country in ['brazil', 'india', 'south_africa']:
        print(f"\n{country.upper()}:")

        # Performance metrics
        performance = simulation_results[country]['performance_metrics']
        best_strategy = performance['Sharpe Ratio'].idxmax()
        worst_strategy = performance['Sharpe Ratio'].idxmin()

        print(f"  Best Strategy: {best_strategy}")
        print(f"    Sharpe Ratio: {performance.loc[best_strategy, 'Sharpe Ratio']:.4f}")
        print(f"    Annual Return: {performance.loc[best_strategy, 'Annual Return']:.4f}")
        print(f"    Max Drawdown: {performance.loc[best_strategy, 'Max Drawdown']:.4f}")

        print(f"  Worst Strategy: {worst_strategy}")
        print(f"    Sharpe Ratio: {performance.loc[worst_strategy, 'Sharpe Ratio']:.4f}")
        print(f"    Annual Return: {performance.loc[worst_strategy, 'Annual Return']:.4f}")
        print(f"    Max Drawdown: {performance.loc[worst_strategy, 'Max Drawdown']:.4f}")

        # Regime transitions
        print(f"  Regime Transitions:")
        for metric, value in transition_analysis[country].items():
            print(f"    {metric}: {value:.4f}" if isinstance(value, float) else f"    {metric}: {value}")

    # Model comparison
    print("\n3. MODEL COMPARISON")
    print("-"*80)

    model_performance = {}

    for country, results in simulation_results.items():
        performance = results['performance_metrics']

        for model in ['RandomForest', 'XGBoost', 'SVC', 'SimpleRNN', 'LSTM', 'GRU']:
            with_macro = f"{model}_with_macro"
            without_macro = f"{model}_without_macro"

            if with_macro in performance.index and without_macro in performance.index:
                if model not in model_performance:
                    model_performance[model] = {
                        'with_macro': {'sharpe': [], 'return': [], 'drawdown': []},
                        'without_macro': {'sharpe': [], 'return': [], 'drawdown': []}
                    }

                model_performance[model]['with_macro']['sharpe'].append(performance.loc[with_macro, 'Sharpe Ratio'])
                model_performance[model]['with_macro']['return'].append(performance.loc[with_macro, 'Annual Return'])
                model_performance[model]['with_macro']['drawdown'].append(performance.loc[with_macro, 'Max Drawdown'])

                model_performance[model]['without_macro']['sharpe'].append(performance.loc[without_macro, 'Sharpe Ratio'])
                model_performance[model]['without_macro']['return'].append(performance.loc[without_macro, 'Annual Return'])
                model_performance[model]['without_macro']['drawdown'].append(performance.loc[without_macro, 'Max Drawdown'])

    for model, perf in model_performance.items():
        print(f"\n{model}:")

        avg_sharpe_with = np.mean(perf['with_macro']['sharpe'])
        avg_sharpe_without = np.mean(perf['without_macro']['sharpe'])
        avg_return_with = np.mean(perf['with_macro']['return'])
        avg_return_without = np.mean(perf['without_macro']['return'])

        print(f"  Average Sharpe Ratio with Macro: {avg_sharpe_with:.4f}")
        print(f"  Average Sharpe Ratio without Macro: {avg_sharpe_without:.4f}")
        print(f"  Improvement: {avg_sharpe_with - avg_sharpe_without:.4f}")

        print(f"  Average Annual Return with Macro: {avg_return_with:.4f}")
        print(f"  Average Annual Return without Macro: {avg_return_without:.4f}")
        print(f"  Improvement: {avg_return_with - avg_return_without:.4f}")

    # Conclusion
    print("\n4. CONCLUSION")
    print("-"*80)

    print("The simulation results demonstrate that market regime classification models")
    print("incorporating macroeconomic factors generally provide improved portfolio performance")
    print("and risk-adjusted returns compared to models using only market data.")

    if avg_sharpe_improvement > 0:
        print(f"\nOn average, including macroeconomic factors improved Sharpe ratios by {avg_sharpe_improvement:.4f}")
        print(f"and annual returns by {avg_return_improvement:.4f} across all models and countries.")
    else:
        print("\nHowever, the improvement was not consistent across all models and countries,")
        print("suggesting that the value of macroeconomic factors may be context-dependent.")

    best_country = macro_value_analysis.groupby('Country')['Sharpe Ratio Improvement'].mean().idxmax()
    print(f"\nThe country that benefited most from macroeconomic factors was {best_country.capitalize()},")
    print("suggesting that market regimes in this emerging market are more strongly")
    print("influenced by macroeconomic conditions.")

    best_model_overall = macro_value_analysis.groupby('Model')['Sharpe Ratio Improvement'].mean().idxmax()
    print(f"\nThe model that most effectively utilized macroeconomic information was {best_model_overall},")
    print("indicating that this approach may be particularly well-suited for integrating")
    print("market and macroeconomic data in regime classification.")

    print("\nThese findings support the thesis that market regime classification incorporating")
    print("macroeconomic factors can contribute to improved portfolio performance and")
    print("risk-adjusted returns for hedge fund managers operating in emerging markets.")

# Generate the report
generate_simulation_report(simulation_results, macro_value_analysis, transition_analysis)
